<a href="https://colab.research.google.com/github/JanEggers-hr/youtube-scraper/blob/main/youtube_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube-Scraper v04

Holt die Videos des übergebenen Youtube-Kanals, isoliert das Audio, und verschriftlicht sie mit einem Speech-to-Text-Service. 

Die einzelnen Schritte:
- Mit youtube_dl nacheinander Videos holen, zu MP3 wandeln
- Die dabei gesammelten wichtigsten Metadaten (Upload-Datum, View-Count) in einer Excel-Datei ablegen
- Mit der OpenAI-Library "Whisper" in Text konvertieren

Dateien liegen danach alle in einem Ordner "output" und müssen von da exportiert werden, weil sie sonst am Ende der Colab-Laufzeit gelöscht werden. Wär doch schade drum. 

## N00bwarnung

Das Skript ist nicht sehr elegant, vor allem aber ist es desaströs langsam. Die youtube_dl-Library lässt sich, soweit ich das erforschen konnte, nicht asynchron ausführen, was schade ist - man könnte ja durchaus mehrere Videos gleichzeitig herunterladen. Vor allem aber könnte man sie Whisper schon mal zum Transkribieren geben. 

So immerhin ist alles schön einfach - wenn auch nicht schnell. 

## GPU einschalten!

Die Whisper-Library profitiert sehr davon, wenn man im Menü unter "Laufzeit/Laufzeittyp ändern" die GPU aktiviert. (Auch wenn Google zunächst meckert, weil das Herunterladen der Videos ohne GPU-Nutzung abgeht.)

Theoretisch könnte man vermutlich auch eine ffmpeg-Variante einbinden, die die GPU nutzt, dann geht der YT-Download schneller... aber: siehe oben. 

## ----
### Changelog
* v04 - Variablen like_count und comment_count in Übersicht aufgenommen
* v03 - Zusammenfassungen über Aleph Alpha und GPT-3 integriert; Sortierung aufsteigend nach Datum
* v02 - Fehler beim Download automatisch auffangen (ganz simpel: Download nochmal starten)
* v01 - Suche nach noch nicht heruntergeladenen Videos; Vervollständigung
* v00 - Funktioniert

### Todo: Mögliche Verbesserungen

- Publikationsdatum der Videos in den Dateinamen, um besseren Überblick zu haben


Hier den Kanal eintragen, der gescraped werden soll - und das Zielverzeichnis.

In [ ]:
channel_url = "https://www.youtube.com/@AudioPilz"
output_dir = "/content/gdrive/MyDrive/youtube-scraper/output"

In [ ]:
# Vorbereitung: youtube_dl installieren
!pip install youtube_dl

In [ ]:
# Für die Datensicherung: Drive verbinden
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Ausgabeverzeichnis output_dir anlegen: 
if not os.path.exists("/content/gdrive/MyDrive/youtube-scraper"):
    os.mkdir("/content/gdrive/MyDrive/youtube-scraper")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

os.chdir(output_dir)

Mit der Library youtube_dl wird eine Liste der Videos mit Metadaten als Tabelle erstellt. Das wird vom Download getrennt, um Abbrüche auffangen zu können - manchmal scheitert youtube_dl an einem "403 FORBIDDEN" der Plattform. 

Also: in der ersten Runde nur Daten sammeln - und als XLSX exportieren. 

In [ ]:
from __future__ import unicode_literals
import youtube_dl
import pandas as pd

# Die Optionen, um neben den Metadaten gleich alle MP3-Dateien herunterzuladen: 
ydl_opts = { 'quiet': 'True' }

# Erste Aufgabe: Hole Metadaten für einen Channel, keine Downloads
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    metadata = ydl.extract_info(channel_url, download=False) 

# Die Daten sind ein bisserl verschachtelt: Die URLs der Videos sind als Dictionary in einer Liste von Dictionaries oder so. Das hier funktioniert
videos_df = pd.DataFrame(metadata['entries'][0]['entries'], columns=["id","upload_date","description","duration","view_count","like_count",
                                                                      "average_rating",
                                                                     "age_limit","categories","tags"])

# Liste aufsteigend nach Datum sortieren und exportieren
videos_df.sort_values("upload_date")
videos_df.to_excel("video_liste.xlsx")
print(len(videos_df)," Videos in der Playlist/Kanal-Startseite gefunden.")
videos_df.head(5)

## Videos herunterladen

*ACHTUNG*: Dieser Schritt dauert eine Weile - und bricht gern mal mit einem Fehler ab, weil Youtube dem youtube-dl-Skript gerne mal ein "Darfste nicht!" in den Weg wirft. **Falls youtube-dl mit einem Fehler abbricht, ruft die Funktion sich selbst noch mal neu auf.** Das stößt (zum Glück) irgendwann an Grenzen - bei zu vielen Rekursionen bricht Python ab. 

Dummerweise verliert das Colab-Notebook nach einer Zeit die virtuelle Maschine, und man muss alles nochmal von vorn starten - deshalb schaut der Code, welche Videos schon heruntergeladen sind, und macht da weiter, wo es zuletzt aufgehört hat. 

In [ ]:
# Definiere den Download als Funktion - die sich im Fehlerfall rekursiv neu aufruft

def download_mp3(videos_liste):
    # Die Optionen, um neben den Metadaten gleich alle MP3-Dateien herunterzuladen: 
    ydl_opts = {'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '128',
    }],
    'outtmpl': '%(id)s.%(ext)s', # Formatiere Dateinamen: id.mp3
    }

    # Leere Liste anlegen
    new_urls = []
    # Videos, für die es noch kein mp3 gibt, in die Liste
    for id in videos_liste:
        f = path + "/" + id + ".mp3"
        if not os.path.exists(f):
            new_urls.append(id)

    if len(new_urls) > 0:
        print("Noch ",len(new_urls)," Videos herunterladen...")
        # Die Liste an den Downloader verfüttern.
        # Videos werden nach dem Download in MP3 gewandelt, das
        # bestimmen die Parameter. 
        try: 
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                    ydl.download(new_urls)
        except:
            # Fehler geworfen; versuch es nochmal
            print("Versuche es nochmal...")
            download_mp3(videos_liste)

    print("Alle Videos des Kanals heruntergeladen!")
    return(True)

# Jetzt die Funktion ausführen
download_mp3(videos_df["id"])

Die Audios liegen alle als MP3 im Ordner output - mit den Dateinamen (id).mp3. Jetzt alle an den STT-Konverter schicken. 

Wir versuchen hier an dieser Stelle mal, OpenAIs "Whisper" einzusetzen. 

Quelle: https://github.com/openai/whisper

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

In [ ]:
pd.read_excel("video_liste.xlsx",index_col=0).head(5)

Wenn die Installation der Library von Github geklappt hat, ist die eigentliche Transkription ziemlich simpel: 

In [ ]:
import whisper
import pandas as pd
model = whisper.load_model("medium")

# Index-Datei nochmal holen
videos_df = pd.read_excel("video_liste.xlsx",index_col=0)

# Wie oben: Liste von allen noch nicht konvertierten Dateien
new_urls = []
for id in videos_df["id"]:
    f = path + "/" + id + "_transcribe.txt"
    if not os.path.exists(f):
        new_urls.append(id)
i = 0
print(len(new_urls)," MP3-Dateien zu verschriftlichen.")

for id in new_urls:
    mp3_fname = path + "/" + id + ".mp3"
    txt_fname = path + "/" + id + "_transcribe.txt"

    result = model.transcribe(mp3_fname)
    # Ergebnis der Umwandlung als Textdatei ausgeben
    with open(txt_fname, 'w') as f:
      f.write(result["text"])
    i = i + 1
    print(i," - ",txt_fname," erzeugt")
    

print("Fertig - ",len(new_urls)," Dateien konvertiert.")

# KI-generierte Nachbearbeitung und Zusammenfassung

Jetzt noch: eine Summary erstellen. 

KI-Sprachmodelle können die Textdatei in einzelne Absätze aufteilen und eine semantische Zusammenfassung erstellen. Das passiert hier - der Service kostet allerdings ein paar Zehntel Cent pro Datei.

### Genutzter KI-Service: Aleph Alpha Luminous Extreme

Das Skript nutzt ein Sprachmodell des deutschen Startups Aleph Alpha, um eine Zusammenfassung schreiben zu lassen. Das Zugangs-Token muss im Google-Drive liegen, im MyDrive-Wurzelverzeichnis in einer Textdatei namens ```aleph_alpha_key.txt```.

Das Aleph-Alpha-Modell 'Luminous Extreme' bietet eine dedizierte Zumsammenfassungs-Funktion. Es ist eingestandenermaßen kleiner und weniger leistungsfähig als das größte GPT3-Modell Davinci; so spart man sich allerdings, einen Cloud-Dienst von OpenAI einzusetzen (was man natürlich auch kann; siehe unten). You're not paying the Man like this.

### Was das Summarization-Modell macht

Da die Textlänge begrenzt ist - Luminous hat eine Maximalgröße von 2048 Tokens, was etwa 600-700 Zeichen entspricht - arbeitet die Summary-Funktion mit einem gleitenden Fenster, das aber deutlich kleiner ist - vielleicht 400 Token (vermute ich). Das Modell reduziert Abschnitte auf einzelne Aufzählungspunkte; die Textmenge wird dabei etwa auf ein Drittel reduziert. 

### Andere Modelle machen es teurer, aber nur wenig besser

Aus diesem Code in ein anderes Skript ausgelagert sind Experimente mit anderen Modellen: Kann man die Vervollständigung von Sprachmodellen wie GPT3-DaVinci oder dem größten Aleph-Alpha-Modell Luminous Supreme nutzen, um bessere Summaries zu erstellen?

Dazu muss man dem Sprachmodell in der Regel ein Beispiel geben. Die Länge des nutzbaren Textfensters schrumpft also auf ca. 800 bzw. 2000 Zeichen. Außerdem zeigen erste Erfahrungen, dass es qualitativ erst dann deutlich besser wird, wenn man einen "Best of 3"-Ansatz wählt. Das Ganze ist also für große Video-Kanäle durchaus eine teure Angelegenheit. 

Deshalb - und weil die qualitative Verbesserung zwar schön ist, aber nicht lebensnotwendig - reicht an dieser Stelle die Luminous-Summary aus. 

In [ ]:
# Library holen und installieren
!pip install aleph_alpha_client

Als erstes das Token aus der Datei ```aleph_alpha_key.txt``` laden und eine Prüfsumme ausgeben. 

Dann die Files durch die KI-Zusammenfassung schicken und diese in die Summary. 

Ein wenig Experimentieren hat gezeigt: Das Modell arbeitet den Text durch und kondensiert ihn in Bullet Points, von denen nicht alle wirklich dem Text entsprechen. Experimentell wählen wir die Bulletpoints aus, die die größte semantische Ähnlichkeit mit dem Gesamttext haben. 

In [ ]:
# Falls das Colab inzwischen alles vergessen hat: 
# Alle Imports nochmal machen; Google-Drive nochmal mounten

import hashlib
import os
import pandas as pd
from google.colab import drive

# Hilfsfunktion: Textdatei wieder einlesen
def gettext(fname):
    try: 
        textfile = open(fname,'r')
    except:
        print("**Datei ",fname," nicht gefunden!**")
        return("")
    text = textfile.readline()
    textfile.close()
    return(text.replace("\n",""))

drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/youtube-scraper/output"
os.chdir(path)

# Erst das Aleph-Alpha-Token holen
aa_token = gettext('/content/gdrive/MyDrive/aleph_alpha_key.txt')

# Den Key gleich nutzen, um die Modelle zu laden
# Boilerplate-Code für Aleph Alpha von https://github.com/Aleph-Alpha/examples/ kopiert
from aleph_alpha_client import AlephAlphaModel, SummarizationRequest, EvaluationRequest, Document

model = AlephAlphaModel.from_model_name(model_name="luminous-extended", token = aa_token)

print("AlephAlpha Token (MD5) ", hashlib.md5(aa_token.encode('utf-8')).hexdigest()," geladen und getestet.")

# Funktion generiert eine Zusammenfassung mit dem Aleph-Alpha-Modell luminous-extended (etwa wie GPT3-Curie.)
def generate_summary(id: str):
    text = gettext(path + "/" + id + "_transcribe.txt")
    request = SummarizationRequest(document=Document.from_text(text))
    result = model.summarize(request)
    print(text[:60],"... zusammengefasst in ",len(result.summary)," Zeichen")
    return result.summary

# Index-Datei nochmal holen
videos_df = pd.read_excel("video_liste.xlsx",index_col=0)
videos_df.sort_values("upload_date",ascending=True)

# Allen Index-Dateizeilen Summaries geben
videos_df["summary"] = videos_df["id"].map(generate_summary)

videos_df.head(10)
videos_df.to_excel("video_liste_annotiert.xlsx")